In [1]:
!pip install xgboost

In [0]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBRegressor
from sklearn import metrics

# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# import matplotlib.pylab as plt

import random

In [3]:
from google.colab import files
uploaded = files.upload()

Saving test2_PCA.csv to test2_PCA.csv
Saving train2_PCA.csv to train2_PCA.csv


In [4]:
test_kaggle = pd.read_csv("test2_PCA.csv")
data_train = pd.read_csv("train2_PCA.csv")

print(test_kaggle.head())
print(data_train.head())

     ID        V1        V2        V3  ...       V97       V98       V99      V100
0  1461 -3.870105 -2.976089  2.481908  ... -0.130741 -0.235667 -0.723380  0.369135
1  1462 -1.417951 -1.695375  2.568001  ... -1.638344  0.995329  0.383772 -1.223461
2  1463  2.815513 -0.461840  0.488803  ...  0.255148 -0.061483 -0.125947  0.381860
3  1464  3.132663  0.136076  0.093511  ...  0.046714  0.285807 -0.510139  0.295245
4  1465  3.024360 -1.352742  0.591631  ... -0.290534 -0.318159  1.116213 -0.821392

[5 rows x 101 columns]
   ID        V1        V2        V3  ...       V98       V99      V100     target
0   1  3.906535  0.087164 -0.814758  ...  0.173181 -0.892435  0.249705  12.247694
1   2  0.325215 -1.308735  1.235708  ...  1.221896 -0.276862  0.178981  12.109011
2   3  4.778343  0.169840 -0.474847  ...  0.526032 -0.223232  0.376139  12.317167
3   4  0.028148 -0.435658 -0.655274  ...  0.320880 -0.053600  0.065094  11.849398
4   5  6.873159  1.384580 -0.666432  ...  0.405929  1.105461 -0.2879

**Sequentially Divided training dataset**

Numpy Array

In [5]:
data_train_array = np.array(data_train.drop('ID', axis=1))
print(data_train_array.shape)
X_train = data_train_array[:, :-1]
print(X_train.shape)
Y_train = data_train_array[:,-1]
print(Y_train.shape)

nfolds = 5
test_CV_N = (X_train.shape[0] // nfolds) # ceiling

(1437, 101)
(1437, 100)
(1437,)


**RandomSearch Parameter Setting**

In [0]:
LearningRate_Max, LearningRate_Min = 1, 0.1
LearningRate_Step = 0.1
LearningRate_N = (LearningRate_Max - LearningRate_Min) / LearningRate_Step + 1 # the number of given learning rates
LearningRate = np.arange(LearningRate_Min, LearningRate_Max+LearningRate_Step, LearningRate_Step) # (0.1, 1, 0.1)

nEstimators = np.arange(0, 5000+100, 100)
MaxDepth = np.arange(1, 9, 2)
MinChildWeight = np.arange(1, 9, 2)

ParamDict = {"learning_rate": random.choice(LearningRate), "n_estimators": random.choice(nEstimators),
             "max_depth": random.choice(MaxDepth), "min_child_weight": random.choice(MinChildWeight)}
print(ParamDict)

{'learning_rate': 0.1, 'n_estimators': 2300, 'max_depth': 7, 'min_child_weight': 7}


In [0]:
RandonSearch_N = 200# must be smaller than LearningRate_N*nEstimators_N*MaxDepth_N*MinChildWeight_N

**Complete Code for RandomSearch**

In [0]:
Result = {}
while RandonSearch_N > 0:
    RandonSearch_N -= 1
    print(RandonSearch_N)
    ParamDict = {"learning_rate": random.choice(LearningRate), "n_estimators": random.choice(nEstimators),
                "max_depth": random.choice(MaxDepth), "min_child_weight": random.choice(MinChildWeight)}
    if ParamDict in Result.values():
        continue
    else:           
        rmse_CV = 0
        for i in range(nfolds):

            ## get train and test data in CV
            test_CV_idx = [j + i*test_CV_N for j in range(test_CV_N)]
            test_CV_x, test_CV_y = X_train[test_CV_idx, :], Y_train[test_CV_idx]
            train_CV_x, train_CV_y = np.delete(X_train, test_CV_idx, axis=0), np.delete(Y_train, test_CV_idx)

            ## train xgb model
            xgb_CV = XGBRegressor(
                learning_rate = ParamDict["learning_rate"],
                n_estimators = ParamDict["n_estimators"],
                max_depth = ParamDict["max_depth"], 
                min_child_weight = ParamDict["min_child_weight"],
                objective='reg:squarederror', 
                subsample=0.8, 
                colsample_bytree=0.8)
            xgb_CV.fit(train_CV_x, train_CV_y)

            #calculate rmse
            rmse_CV += np.sqrt(metrics.mean_squared_error(xgb_CV.predict(test_CV_x), test_CV_y))

        Result[rmse_CV / nfolds] = ParamDict

In [0]:
Result_sorted = {k: v for k, v in sorted(Result.items(), key = lambda x:x[0])}
Result_df = pd.DataFrame(Result_sorted)
pd.DataFrame(Result_sorted)

,0.130882,0.133010,0.133011,0.133085,0.133115,0.133130,0.133498,0.134352,0.134370,0.134377,0.134645,0.134647,0.134960,0.134971,0.134972,0.135210,0.135230,0.135529,0.135882,0.136489,0.136498,0.136608,0.137171,0.137829,0.137831,0.137967,0.137989,0.138036,0.138284,0.138428,0.138452,0.138804,0.139201,0.139575,0.139642,0.139785,0.140689,0.141826,0.142263,0.142679,...,0.207854,0.212278,0.214581,0.214727,0.214752,0.221027,0.221114,0.221118,0.221410,0.227432,0.237081,0.242622,0.242623,0.243720,0.245773,0.248981,0.250100,0.250124,0.250151,0.251330,0.251588,0.252997,0.256050,0.256056,0.264890,0.265075,0.271711,0.271864,0.276977,0.282479,0.282486,0.282487,0.286042,0.286076,0.288197,0.308676,0.308761,0.318539,0.321025,11.526796
learning_rate,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.2,0.2,0.2,0.1,0.1,0.1,0.1,0.2,0.2,0.2,0.1,0.1,0.2,0.2,0.2,0.2,0.3,0.1,0.3,0.3,0.2,0.3,0.3,0.3,0.3,...,0.8,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.8,0.8,1.0,1.0,1.0,0.8,1.0,0.8,0.8,0.8,1.0,0.8,0.9,0.9,0.9,0.9,0.9,1.0,1.0,1.0,1.0,1.0,1.0,0.9,0.9,0.9,1.0,1.0,1.0,1.0,0.9
n_estimators,1500.0,2900.0,1700.0,800.0,2400.0,4700.0,800.0,1900.0,4900.0,2700.0,1000.0,4500.0,800.0,3500.0,3700.0,400.0,3400.0,500.0,1800.0,4500.0,2700.0,4900.0,100.0,1600.0,1500.0,2400.0,1100.0,900.0,700.0,3500.0,1000.0,200.0,3600.0,3400.0,1600.0,5000.0,200.0,4000.0,3500.0,700.0,...,4100.0,900.0,1500.0,4700.0,4300.0,1900.0,2100.0,3300.0,3700.0,4000.0,2600.0,3700.0,3400.0,3800.0,1500.0,3600.0,2700.0,3100.0,1500.0,1600.0,900.0,1700.0,2800.0,400.0,3000.0,1500.0,4300.0,300.0,2800.0,1500.0,4800.0,3000.0,3200.0,300.0,500.0,3600.0,4500.0,200.0,2400.0,0.0
max_depth,3.0,5.0,5.0,3.0,3.0,3.0,7.0,5.0,5.0,5.0,5.0,5.0,7.0,7.0,7.0,3.0,3.0,1.0,1.0,7.0,7.0,5.0,7.0,3.0,3.0,7.0,7.0,1.0,5.0,5.0,5.0,1.0,1.0,3.0,3.0,5.0,3.0,3.0,5.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,5.0,3.0,3.0,3.0,7.0,3.0,7.0,7.0,7.0,3.0,7.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,3.0
min_child_weight,3.0,5.0,5.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,7.0,7.0,7.0,7.0,7.0,1.0,1.0,3.0,1.0,3.0,3.0,3.0,7.0,7.0,7.0,1.0,1.0,5.0,7.0,1.0,1.0,7.0,7.0,3.0,3.0,3.0,1.0,5.0,5.0,3.0,...,1.0,7.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,5.0,3.0,5.0,1.0,3.0,3.0,3.0,7.0,1.0,5.0,7.0,7.0,1.0,1.0,3.0,3.0,5.0,7.0,7.0,7.0,5.0,5.0,7.0,3.0,3.0,1.0,5.0,5.0


In [0]:
Result_df.to_csv("RandomSearch v1_200.csv")
Result_df.to_csv("RandomSearch v1_"+str(Result_df.shape[1])+".csv")
# 10%

# nEstimators, we have three cutoff: 600, 1500, 2100
# [1500-300, 1500+300, 30] 20%, step = 2%; 40%, step = 2% n_estimate: range is broad
# [2, 5, 1]
# [2, 5, 1]

# [0.01, 0.1, 0.01]
# [100, 600, 10]
# [2, 4, 1]
# [2, 4, 1]

**GridSearch Parameter Setting**

In [0]:
# WRONG!!!!---------------------------------------------------------------
# LearningRate_GS = np.arange(0.01, 0.1+0.01, 0.01) # [0.01, 0.1, 0.01]
# nEstimators_GS = np.arange(1200, 1800+30, 30) # [1500-300, 1500+300, 30] 20%, step = 2%;
# MaxDepth_GS = np.arange(2, 5+1, 1) # [2, 5, 1]
# MinChildWeight_GS = np.arange(2, 5+1, 1) # [2, 5, 1]

# GridSearch_N = len(LearningRate_GS) * len(nEstimators_GS) * len(MaxDepth_GS) * len(MinChildWeight_GS)
# LearningRateComb = np.tile(LearningRate_GS, GridSearch_N//len(LearningRate_GS))
# nEstimatorsComb = np.tile(nEstimators_GS, GridSearch_N//len(nEstimators_GS))
# MaxDepthComb = np.tile(MaxDepth_GS, GridSearch_N//len(MaxDepth_GS))
# MinChildWeightComb = np.tile(MinChildWeight_GS, GridSearch_N//len(MinChildWeight_GS))


In [0]:
# print(len(LearningRateComb), len(nEstimatorsComb), len(MaxDepthComb), len(MinChildWeightComb))
# print([LearningRateComb, nEstimatorsComb, MaxDepthComb, MinChildWeightComb])

3360 3360 3360 3360
[array([0.01, 0.02, 0.03, ..., 0.08, 0.09, 0.1 ]), array([1200, 1230, 1260, ..., 1740, 1770, 1800]), array([2, 3, 4, ..., 3, 4, 5]), array([2, 3, 4, ..., 3, 4, 5])]


In [0]:
LearningRate_GS = np.arange(0.01, 0.1+0.01, 0.01) # [0.01, 0.1, 0.01]
nEstimators_GS = np.arange(1200, 1800+30, 30) # [1500-300, 1500+300, 30] 20%, step = 2%;
MaxDepth_GS = np.arange(2, 5+1, 1) # [2, 5, 1]
MinChildWeight_GS = np.arange(2, 5+1, 1) # [2, 5, 1]
GridSearch_N = len(LearningRate_GS) * len(nEstimators_GS) * len(MaxDepth_GS) * len(MinChildWeight_GS)

count = 0
ParamComb = {}
for lr in LearningRate_GS:
    for n in nEstimators_GS:
        for d in MaxDepth_GS:
            for c in MinChildWeight_GS:
                ParamComb[count] = {"learning_rate": lr, "n_estimators": n, "max_depth": d, "min_child_weight": c}
                count += 1

In [0]:
print(len(ParamComb))
sorted(ParamComb.items(), key = lambda x:(x[1]["learning_rate"], x[1]["n_estimators"], x[1]["max_depth"]))

**Complete Code for GridSearch**

In [0]:
GridSearch_N = 300


In [14]:
print(GridSearch_N)

300


In [15]:

Result_GS = {} # store GridSearch CV result
Param_GS = {}
while GridSearch_N > 0:
    GridSearch_N -= 1
    print(GridSearch_N)

    ParamDict = {"learning_rate": ParamComb[GridSearch_N]["learning_rate"], "n_estimators": ParamComb[GridSearch_N]["n_estimators"],
                "max_depth": ParamComb[GridSearch_N]["max_depth"], "min_child_weight": ParamComb[GridSearch_N]["min_child_weight"]}
           
    rmse_CV = 0
    for i in range(nfolds):
        ## get train and test data in CV
        test_CV_idx = [j + i*test_CV_N for j in range(test_CV_N)]
        test_CV_x, test_CV_y = X_train[test_CV_idx, :], Y_train[test_CV_idx]
        train_CV_x, train_CV_y = np.delete(X_train, test_CV_idx, axis=0), np.delete(Y_train, test_CV_idx)

        ## train xgb model
        xgb_CV = XGBRegressor(
            learning_rate = ParamDict["learning_rate"],
            n_estimators = ParamDict["n_estimators"],
            max_depth = ParamDict["max_depth"], 
            min_child_weight = ParamDict["min_child_weight"],
            objective='reg:squarederror', 
            subsample=0.8, 
            colsample_bytree=0.8)
        xgb_CV.fit(train_CV_x, train_CV_y)
        #calculate rmse
        rmse_CV += np.sqrt(metrics.mean_squared_error(xgb_CV.predict(test_CV_x), test_CV_y))
    Param_GS[GridSearch_N] = rmse_CV / nfolds
    Result_GS[rmse_CV / nfolds] = ParamDict

299
298
297
296
295
294
293
292
291
290
289
288
287
286
285
284
283
282
281
280
279
278
277
276
275
274
273
272
271
270
269
268
267
266
265
264
263
262
261
260
259
258
257
256
255
254
253
252
251
250
249
248
247
246
245
244
243
242
241
240
239
238
237
236
235
234
233
232
231
230
229
228
227
226
225
224
223
222
221
220
219
218
217
216
215
214
213
212
211
210
209
208
207
206
205
204
203
202
201
200
199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33

In [16]:
rmse = list(Result_GS.keys())
len(rmse)

300

In [17]:
Result_GS_sorted = {k: v for k, v in sorted(Result_GS.items(), key = lambda x:x[0])}
Result_GS_df = pd.DataFrame(Result_GS_sorted)
print(pd.DataFrame(Result_GS_sorted))

                  0.128909  0.128923  0.128931  ...  0.134278  0.134438  0.134590
learning_rate         0.01      0.01      0.01  ...      0.01      0.01      0.01
n_estimators       1740.00   1740.00   1740.00  ...   1230.00   1200.00   1200.00
max_depth             3.00      3.00      4.00  ...      2.00      2.00      2.00
min_child_weight      4.00      5.00      3.00  ...      2.00      3.00      2.00

[4 rows x 300 columns]


In [0]:
Result_GS_df.to_csv("GridSearch v1_"+str(Result_GS_df.shape[1])+".csv")

In [0]:
Result_GS_df.to_csv("GridSearch v1_"+"0-300"+".csv")